In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
VOCAB_SIZE = 5000
SEQUENCE_LENGTH = 1200

Brass set described in the paper
1) descriptions that are very long (> 750 characters) or short (< 20 characters)
2) descriptions with high lexical overlap (over 40%) with their show description
3) descriptions with high lexical overlap (over 50%) with other episode descriptions

In [34]:
def brass_length(dataset, tokenizer, upp_bound=750, low_bound=20):
    """
    Returns a dataset with less samples removing ones having descriptions that are very long (> 750 characters)
    or short (< 20 characters)
    Parameters:
        - dataset: pandas dataframe containing transcript and summary for each podcast
        - tokenizer: tokenizer to use for tokenize the description
        - upp_bound: maximum number of characters in the description
        - low_bound: minimum number of characters in the description
    Returns:
        - subset of the dataset applying the aforementioned filter
    """
    rows = dataset.shape[0]
    for i in range(rows):
        tokenized_description = tokenizer(dataset['episode_description'][i])
        tokenized_description = np.nonzero(tokenized_description.numpy())[0]
        if len(tokenized_description) > upp_bound or len(tokenized_description) < low_bound:
            dataset.drop([i], inplace=True)

    return dataset

# testing
data = [["transcript one", "this is the first summary of a podcast. I need to reach at least twenty " \
        "characters. It is not enough, I need to add other characters."], ['transcript 2', 'too short, it must be removed']]

vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=SEQUENCE_LENGTH)
vectorizer.adapt(np.array(data).flatten())

df = pd.DataFrame(data, columns=['episode_transcript', 'episode_description'])

brass_length(df, vectorizer)


,transcript,description
0,transcript one,this is the first summary of a podcast. I need...


In [3]:
def brass_overlap_show(dataset, tokenizer, percentage=0.4):
    """
    Delete from the dataset podcast having a description which overlaps over percentage the show description
    Parameters:
        - dataset: dataset: pandas dataframe containing transcript and summary for each podcast
        - tokenizer: tokenizer to use for tokenize the description and the show description
        - percentage: threshold to discard the podcast
    Returns:
        - cleaned dataset
    """
    rows = dataset.shape[0]
    for i in range(rows):
        tokenized_description = tokenizer(dataset['episode_description'][i]).numpy()
        tokenized_show = tokenizer(dataset['show_description'][i]).numpy()
        # index of meaningful tokens
        index_description = np.nonzero(tokenized_description)[0]
        index_show = np.nonzero(tokenized_show)[0]
        # meaningful tokens
        description = tokenized_description[index_description]
        show = tokenized_show[index_show]
        # if the shortest sequence is a small fraction wrt the percentage threshold and the description
        # it is possible to be sure it will not overlap
        if len(show) / len(description) <= percentage:
            continue
        else:
            # comparison of all possible matches
            j = 0
            while j <= len(description) - len(show):
                # comparison between episode description and show description
                common_index = (description[j:j+len(show)] == show)
                overlap = len(common_index) / len(description)
                if overlap > percentage:
                    dataset.drop([i], inplace=True)
                    j = len(description) - len(show) + 1
                else:
                    j += 1
    return dataset

# Testing
data = [["this is gonna be an overlapping", "this is gonna be an overlapping, trust me"],
        ["this will not be an overlapping", "this will not be an overlapping sentence because the content is much different between the two descriptions"]]

vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=SEQUENCE_LENGTH)
vectorizer.adapt(np.array(data).flatten())

df = pd.DataFrame(data, columns=['show_description', 'episode_description'])

brass_overlap_show(df, vectorizer)

,show_description,episode_description
1,this will not be an overlapping,this will not be an overlapping sentence becau...


In [ ]:
def brass_overlap_descriptions(dataset, tokenizer, percentage=0.5):
    """
    Delete from the dataset podcast having a description which overlaps over percentage another description
    Parameters:
        - dataset: dataset: pandas dataframe containing transcript and summary for each podcast
        - tokenizer: tokenizer to use for tokenize the descriptions
        - percentage: threshold to discard the podcast
    Returns:
        - cleaned dataset
    """
    rows = dataset.shape[0]
    deleted_podcast_idx = set()
    for i in range(rows):
        # if the podcast has been already excluded, do not consider it
        if i in deleted_podcast_idx:
            continue
        else:
            for j in range(i+1, rows):
                if j in deleted_podcast_idx:
                    continue
                else:
                    tokenized_description_one = tokenizer(dataset['episode_description'][i]).numpy()
                    tokenized_description_two = tokenizer(dataset['episode_description'][j]).numpy()
                    # index of meaningful tokens
                    index_one = np.nonzero(tokenized_description_one)[0]
                    index_two = np.nonzero(tokenized_description_two)[0]
                    # meaningful tokens
                    description_one = tokenized_description_one[index_one]
                    description_two = tokenized_description_two[index_two]
                    # comparison between episode descriptions
                    n = min(len(description_one), len(description_two))
                    common_index = (description_one[:n] == description_two[:n])
                    overlap = len(common_index) / max(len(description_one), len(description_two))
                    if overlap > percentage:
                        deleted_podcast_idx.add(j)

    dataset.drop(list(deleted_podcast_idx), inplace=True)
    return dataset

# Testing
        
